# Ejercicio 03: Cálculo de Relevancia y Ranking de Documentos

El objetivo de este ejercicio es calcular analíticamente la relevancia de cada documento en un corpus y luego ordenar (rankear) los documentos basándonos en esa puntuación de relevancia para tres consultas específicas. Seguirás los siguientes pasos:

Descripción del Ejercicio

1. Procesamiento del Corpus:
    * Leer y parsear el archivo XML proporcionado que contiene el corpus de documentos con sus metadatos (keywords, autor y fecha).

2. Procesamiento de las Consultas:
    * Definir las consultas proporcionadas.
    * Extraer las palabras clave de cada consulta.

3. Cálculo de Relevancia:
    * Utilizar métricas de similitud (Similitud Coseno y Jaccard) entre la representación vectorial de los documentos y las de las consultas.
    * Calcular la puntuación de relevancia para cada documento del corpus respecto a cada consulta.

4. Ranking de Documentos:
    * Ordenar los documentos en función de su puntuación de relevancia de mayor a menor.
    * Mostrar el ranking de documentos para cada consulta.

### Procedimiento
#### 1. Procesamiento del corpus

In [1]:
# Importar las librerías necesarias
import xml.etree.ElementTree as ET
import re
from math import sqrt
import math

In [2]:
# Definir las consultas
queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}

In [3]:
# Función para procesar el texto y extraer palabras clave
def process_text(text):
    # Convertir a minúsculas
    text = text.lower()
    
    # Reemplazar caracteres no alfanuméricos por espacios
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    
    # Tokenizar y eliminar palabras vacías si es necesario
    tokens = text.strip().split()
    return set(tokens)

In [4]:
# Paso 1: Leer y parsear el archivo XML
def parse_corpus(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    corpus = {}
    for doc in root.findall('document'):
        doc_id = int(doc.get('id'))
        title = doc.find('title').text
        keywords = doc.find('keywords').text
        author = doc.find('author').text
        date = doc.find('date').text
        
        # Procesar las palabras clave
        keyword_set = process_text(keywords)
        corpus[doc_id] = {
            'title': title,
            'keywords': keyword_set,
            'author': author,
            'date': date
        }
    return corpus

In [5]:
# Ejecutar la función con el archivo proporcionado
corpus = parse_corpus('../data/03ranking_corpus.xml')
for doc_id, doc_data in corpus.items():
    print(f"Documento ID: {doc_id}")
    for key, value in doc_data.items():
        print(f"  {key.capitalize()}: {value}")
    print("-" * 40)

Documento ID: 1
  Title: El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.
  Keywords: {'médica', 'salud', 'telemedicina', 'tratamiento', 'tecnología', 'crónica'}
  Author: Dr. Juan Pérez
  Date: 2023-01-15
----------------------------------------
Documento ID: 2
  Title: Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
  Keywords: {'mental', 'académico', 'nutrición', 'rendimiento', 'salud', 'estudiantes'}
  Author: Dra. María López
  Date: 2023-02-10
----------------------------------------
Documento ID: 3
  Title: Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.
  Keywords: {'relaciones', 'campus', 'sociales', 'estudiantil', 'amistad', 'bienestar'}
  Author: Miguel Rodríguez
  Date: 2023-03-05
----------------------------------------
Documento ID: 4
  Title: El rol de las bibliotecas universitarias en el fomento de la investigación académica.
  Keyword

### 2. Procesamiento de las Consultas

In [6]:
# 2.1 Definir las consultas proporcionadas
# Inicializa un diccionario vacío para almacenar solo el texto de cada consulta proporcionada
query_texts = {}

# Itera sobre cada consulta en el diccionario 'queries', donde cada clave (qid) es un identificador de consulta
for qid, query in queries.items():
    # Almacena el texto de la consulta en el diccionario 'query_texts' usando su identificador como clave
    query_texts[qid] = query

# Imprime el resultado que contiene todas las consultas proporcionadas, sin procesar
query_texts

{1: 'Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios',
 2: 'Actividades extracurriculares y bienestar emocional en el campus universitario',
 3: 'Estrategias universitarias para reducir el estrés en estudiantes'}

In [7]:
# 2.2 Extraer las palabras clave de cada consulta
# Inicializa un diccionario vacío para almacenar las palabras clave de cada consulta
query_keywords = {}

# Itera a través de cada consulta en el diccionario 'queries' usando el identificador y el texto de la consulta
for qid, query in queries.items():
    
    # Llama a la función 'process_text' para extraer las palabras clave del texto de la consulta
    keyword_set = process_text(query)
    
    # Almacena tanto el texto original de la consulta como su conjunto de palabras clave en el diccionario 'query_keywords'
    query_keywords[qid] = {
        'text': query,
        'keywords': keyword_set
    }

# Imprime los resultados del diccionario 'query_keywords', que contiene cada consulta y sus palabras clave extraídas
query_keywords

{1: {'text': 'Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios',
  'keywords': {'académico',
   'de',
   'el',
   'en',
   'estudiantes',
   'impacto',
   'la',
   'los',
   'mental',
   'rendimiento',
   'salud',
   'universitarios'}},
 2: {'text': 'Actividades extracurriculares y bienestar emocional en el campus universitario',
  'keywords': {'actividades',
   'bienestar',
   'campus',
   'el',
   'emocional',
   'en',
   'extracurriculares',
   'universitario',
   'y'}},
 3: {'text': 'Estrategias universitarias para reducir el estrés en estudiantes',
  'keywords': {'el',
   'en',
   'estrategias',
   'estrés',
   'estudiantes',
   'para',
   'reducir',
   'universitarias'}}}

### 3. Cálculo de Relevancia

In [8]:
# Función para calcular la similitud de coseno entre dos conjuntos de palabras
def cosine_similarity(set1, set2):
    # Crear el conjunto de palabras únicas a partir de la unión de ambos conjuntos
    words = list(set1.union(set2))
    
    # Crear los vectores de frecuencia binarios para cada conjunto
    vec1 = [1 if word in set1 else 0 for word in words]
    vec2 = [1 if word in set2 else 0 for word in words]

    # Calcular el producto punto y las magnitudes de los vectores
    dot_product = sum(v1 * v2 for v1, v2 in zip(vec1, vec2))
    magnitude1 = math.sqrt(sum(v ** 2 for v in vec1))
    magnitude2 = math.sqrt(sum(v ** 2 for v in vec2))
    
    # Evitar la división por cero, en caso de que una magnitud sea cero
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0
    
    # Retorna la similitud de coseno entre los dos conjuntos
    return dot_product / (magnitude1 * magnitude2)

# Función para calcular la similitud de Jaccard entre dos conjuntos de palabras
def jaccard_similarity(set1, set2):
    # Calcular la intersección y la unión de los dos conjuntos
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    
    # Evitar la división por cero en caso de que la unión sea vacía
    if not union:
        return 0.0
    
    # Retorna la similitud de Jaccard como el tamaño de la intersección dividido por el tamaño de la unión
    return len(intersection) / len(union)

In [9]:
# Función para calcular la relevancia de cada documento respecto a cada consulta
def calculate_relevance(corpus, query_keywords):
    # Inicializa un diccionario para almacenar la relevancia de cada documento respecto a cada consulta
    relevance = {}
    
    # Itera sobre cada consulta en 'query_keywords'
    for qid, query_data in query_keywords.items():
        relevance[qid] = {}
        query_kw = query_data['keywords']
        
        # Itera sobre cada documento en el corpus
        for doc_id, doc_data in corpus.items():
            doc_kw = doc_data['keywords']

            # Calcula la similitud de coseno y Jaccard entre las palabras clave de la consulta y el documento
            cos_sim = cosine_similarity(query_kw, doc_kw)
            jac_sim = jaccard_similarity(query_kw, doc_kw)

            # Almacena las similitudes en el diccionario de relevancia para cada consulta y documento
            relevance[qid][doc_id] = {
                'cosine_similarity': cos_sim,
                'jaccard_similarity': jac_sim
            }

    # Retorna el diccionario de relevancia que contiene las puntuaciones de similitud para cada consulta y documento
    return relevance

# Calcula la relevancia de cada documento en el corpus respecto a cada consulta y almacena los resultados
relevance = calculate_relevance(corpus, query_keywords)

# # Imprime el diccionario 'relevance', que contiene las puntuaciones de relevancia calculadas para cada consulta y documento
# relevance

In [10]:
# Formatear los resultados de relevancia para una mejor visualización
def print_relevance_results(relevance):
    for qid, docs in relevance.items():
        print(f"Resultados para la Consulta {qid}:\n")
        for doc_id, scores in docs.items():
            cos_sim = round(scores['cosine_similarity'], 10)
            jac_sim = round(scores['jaccard_similarity'], 10)
            print(f"  Documento {doc_id}:")
            print(f"    - Similitud Coseno: {cos_sim}")
            print(f"    - Similitud Jaccard: {jac_sim}")
        print("\n" + "-" * 40 + "\n")  # Separador entre consultas

# Llamada a la función para imprimir los resultados formateados
print_relevance_results(relevance)

Resultados para la Consulta 1:

  Documento 1:
    - Similitud Coseno: 0.1178511302
    - Similitud Jaccard: 0.0588235294
  Documento 2:
    - Similitud Coseno: 0.589255651
    - Similitud Jaccard: 0.3846153846
  Documento 3:
    - Similitud Coseno: 0.0
    - Similitud Jaccard: 0.0
  Documento 4:
    - Similitud Coseno: 0.0
    - Similitud Jaccard: 0.0
  Documento 5:
    - Similitud Coseno: 0.0
    - Similitud Jaccard: 0.0
  Documento 6:
    - Similitud Coseno: 0.1178511302
    - Similitud Jaccard: 0.0588235294
  Documento 7:
    - Similitud Coseno: 0.589255651
    - Similitud Jaccard: 0.3846153846
  Documento 8:
    - Similitud Coseno: 0.1178511302
    - Similitud Jaccard: 0.0588235294
  Documento 9:
    - Similitud Coseno: 0.1091089451
    - Similitud Jaccard: 0.0555555556
  Documento 10:
    - Similitud Coseno: 0.0
    - Similitud Jaccard: 0.0
  Documento 11:
    - Similitud Coseno: 0.4714045208
    - Similitud Jaccard: 0.2857142857
  Documento 12:
    - Similitud Coseno: 0.25819888

### 4. Ranking de Documentos

In [11]:
# Función para ordenar los documentos en función de su puntuación de relevancia
def rank_results(relevance):
    rankings = {}  # Diccionario para almacenar el ranking de documentos para cada consulta

    for qid, scores in relevance.items():  # Iterar sobre cada consulta y sus puntajes de relevancia
        # Ordenar los documentos por similitud coseno en orden descendente
        ranked_docs = sorted(scores.items(), key=lambda x: x[1]['cosine_similarity'], reverse=True)
        rankings[qid] = ranked_docs  # Guardar el ranking ordenado de documentos para la consulta actual

    return rankings  # Retornar el diccionario con los rankings para todas las consultas

In [12]:
# Llamar a la función para obtener los documentos ordenados por relevancia
ranked_documents = rank_results(relevance)

In [13]:
# Mostrar el ranking de documentos para cada consulta
for qid, ranked_docs in ranked_documents.items():
    print(f"Consulta: {queries[qid]}")  # Imprimir la consulta actual
    for rank, (doc_id, scores) in enumerate(ranked_docs, 1):  # Enumerar y mostrar el ranking
        # Imprimir el ranking y título del documento
        print(f"  Rango {rank}: Documento {doc_id} ({corpus[doc_id]['title']})")
    print("\n" + "-" * 40 + "\n")  # Separador entre consultas para mayor legibilidad

Consulta: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios
  Rango 1: Documento 2 (Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.)
  Rango 2: Documento 7 (La importancia del sueño en la salud mental y el rendimiento académico en jóvenes universitarios.)
  Rango 3: Documento 14 (Cómo el acceso a servicios de salud mental en la universidad puede mejorar el desempeño académico.)
  Rango 4: Documento 13 (Estrategias para mejorar la salud mental en estudiantes universitarios: el rol del apoyo psicológico y los grupos de estudio en el campus.)
  Rango 5: Documento 11 (Impacto de la práctica regular de ejercicio en la reducción del estrés académico en estudiantes universitarios.)
  Rango 6: Documento 19 (Factores de riesgo y estrategias para prevenir el agotamiento académico en estudiantes universitarios.)
  Rango 7: Documento 29 (Análisis de los hábitos de sueño y su relación con el rendimiento en exámene